In [1]:
%pip install nltk


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Akshat Sharma 102217107 3CS4

In [2]:
import nltk
import pandas as pd

In [3]:
df = pd.read_csv("docs/IMDB_Dataset.csv")
# print(df.head())

In [ ]:
corpus=[]
for i in range(100):
    corpus.append(df["review"].iloc[i])
# print(corpus)

In [ ]:
lower=[]
for i in corpus:
    lower.append(' '.join([word.lower() for word in i.split()]))
# print(lower)

In [ ]:
alpha=[]
for i in lower:
    alpha.append(' '.join([word for word in i.split() if word.isalpha()]))
# alpha

In [11]:
tokenize=[]
for i in alpha:
    tokenize.append([word for word in i.split()])
# tokenize

In [13]:
from nltk.corpus import *
stopword=nltk.corpus.stopwords.words('english') #stopword will contain list of all stopwords of english language
no_stop=[]
for i in tokenize:
    no_stop.append([word for word in i if word not in stopword])
# no_stop

In [24]:
final=[] #will contain final pre-processed documents
from nltk.stem import PorterStemmer
ps=PorterStemmer()
for i in no_stop:
    final.append(' '.join([ps.stem(word) for word in i]))
# final

# Binary TDM

In [74]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(binary=True)
X=cv.fit_transform(final) # returns a binary tdm in compressed sparse row format
# print(X)
binary_tdm_inbuilt=pd.DataFrame(X.toarray().T,index=cv.get_feature_names_out(),columns=[f'{i + 1} docs' for i in range(len(final))])
binary_tdm_inbuilt.shape

(2977, 100)

In [71]:
doc_count={} #dictionary for each document
for i in range(len(final)):
    word_count={} #dictionary for words for each document
    for word in final[i].split():
        if word not in word_count.keys():
            word_count[word]=1
    doc_count[f"{i+1} doc"]=word_count
# doc_count

In [73]:
binary_tdm_scratch = pd.DataFrame(doc_count)
binary_tdm_scratch.fillna(0,inplace=True)
binary_tdm_scratch.shape

(2982, 100)

# Term Frequency Matrix

In [78]:
doc_count1={} #dictionary for each document
for i in range(len(final)):
    word_count1={} #dictionary for words for each document
    for word in final[i].split():
        if word not in word_count1.keys():
            word_count1[word]=0
        word_count1[word]+=1
    doc_count1[f"{i+1} doc"]=word_count1
# doc_count1

In [75]:
#Converting dictionary to term frequency TDM dataframe
tdm_tf_scratch=pd.DataFrame(doc_count1)
tdm_tf_scratch.fillna(0,inplace=True)
tdm_tf_scratch.shape

(2982, 100)

In [86]:
from sklearn.feature_extraction.text import CountVectorizer
cv1=CountVectorizer()
Y=cv1.fit_transform(final) # returns a term frequency tdm in compressed sparse row (csr) format 
# print(Y)
tdm_tf_inbuilt=pd.DataFrame(Y.toarray().T,index=cv1.get_feature_names_out(),columns=[f'{i + 1}.txt' for i in range(len(final))])
tdm_tf_inbuilt.shape

(2977, 100)

# TDM Length Normalization

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(use_idf=False,norm='l1')
Z=tf.fit_transform(final)
tdm_len_norm_inbuilt=pd.DataFrame(Z.toarray().T,index=tf.get_feature_names_out(),columns=[f'{i + 1}.txt' for i in range(len(final))])
tdm_len_norm_inbuilt.shape

(2977, 100)

In [63]:
tdm_length_norm_scratch=tdm_tf_scratch.iloc[:,:].div(tdm_tf_scratch.sum(axis=0),axis=1)
tdm_length_norm_scratch.shape

(2982, 100)

# TF IDF

In [56]:
import numpy as np
no_of_docs=len(final)
unique_terms=tdm_length_norm_scratch.index #as idf is computed for each unique term of corpus
idf_count={} #to store idf score of each term of corpus
for term in unique_terms:
    idf_count[term]=0 #initializing entry in idf dictionary
    for doc in final:
        if term in doc.split():
            idf_count[term]+=1
    idf_count[term]=np.log10(no_of_docs/idf_count[term])
idf_series = pd.Series(idf_count)
idf = pd.DataFrame(idf_series)
idf.head()

,0
one,0.309804
review,1.698970
mention,1.221849
watch,0.537602
oz,2.000000


In [79]:
tf_idf_scratch=tdm_length_norm_scratch*idf.values
tf_idf_scratch.head()

,1 doc,2 doc,3 doc,4 doc,5 doc,6 doc,7 doc,8 doc,9 doc,10 doc,...,91 doc,92 doc,93 doc,94 doc,95 doc,96 doc,97 doc,98 doc,99 doc,100 doc
one,0.002560,0.004918,0.004918,0.000000,0.019987,0.009388,0.0,0.005079,0.012645,0.0,...,0.0,0.005341,0.0,0.003972,0.00000,0.0000,0.008488,0.000000,0.004556,0.0
review,0.014041,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,0.0
mention,0.010098,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,0.0
watch,0.013329,0.008533,0.008533,0.010752,0.000000,0.000000,0.0,0.000000,0.010971,0.0,...,0.0,0.009269,0.0,0.000000,0.01344,0.0064,0.000000,0.010143,0.023718,0.0
oz,0.049587,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,0.0


In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf1=TfidfVectorizer()
U=tf1.fit_transform(final)
tf_idf_inbuilt=pd.DataFrame(U.toarray().T,index=tf1.get_feature_names_out(),columns=[f'{i + 1}.txt' for i in range(len(final))])
tf_idf_inbuilt.head()

,1.txt,2.txt,3.txt,4.txt,5.txt,6.txt,7.txt,8.txt,9.txt,10.txt,...,91.txt,92.txt,93.txt,94.txt,95.txt,96.txt,97.txt,98.txt,99.txt,100.txt
abbot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
abbrevi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.147244,0.0,0.0,0.0
abet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.088988,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
abid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
abil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [41]:
co_occur_matrix=np.dot(binary_tdm_scratch.values,binary_tdm_scratch.values.T)
np.fill_diagonal(co_occur_matrix,0)
co_occur_matrix=pd.DataFrame(co_occur_matrix, index=binary_tdm_scratch.index,
                            columns=binary_tdm_scratch.index)
co_occur_matrix.head()

,one,review,mention,watch,oz,episod,exactli,happen,first,thing,...,spin,primari,activ,pointer,enemi,object,gripe,upsid,restart,superb
one,0.0,2.0,2.0,16.0,1.0,2.0,5.0,4.0,13.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
review,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mention,2.0,1.0,0.0,3.0,1.0,1.0,1.0,2.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
watch,16.0,1.0,3.0,0.0,1.0,1.0,1.0,3.0,6.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
oz,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
term_frequency = pd.DataFrame(binary_tdm_scratch.sum(axis=1))
term_frequency_independent = term_frequency.dot(term_frequency.T)
ppmi_scratch = (co_occur_matrix * len(final)) / (term_frequency_independent)
ppmi_scratch = np.log(ppmi_scratch)
ppmi_scratch = np.maximum(ppmi_scratch, 0)
ppmi_scratch.head()

c:\Users\aksha\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


,one,review,mention,watch,oz,episod,exactli,happen,first,thing,...,spin,primari,activ,pointer,enemi,object,gripe,upsid,restart,superb
one,0.000000,0.713350,0.000000,0.118643,0.713350,0.713350,0.713350,0.153734,0.020203,0.020203,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
review,0.713350,0.000000,2.120264,0.544727,3.912023,3.218876,2.302585,1.966113,0.653926,0.653926,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mention,0.000000,2.120264,0.000000,0.544727,2.813411,2.120264,1.203973,1.560648,0.653926,0.248461,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
watch,0.118643,0.544727,0.544727,0.000000,1.237874,0.544727,0.000000,0.390576,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
oz,0.713350,3.912023,2.813411,1.237874,0.000000,3.912023,2.995732,2.659260,1.347074,1.347074,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
